In [ ]:
import keras.backend as K
import tensorflow as tf
from tensorflow.python import saved_model
from tensorflow.python.saved_model.signature_def_utils_impl import (
    build_signature_def, predict_signature_def
)
from keras_retinanet import models
import shutil
import os

os.environ['CUDA_VISIBLE_DEVICES'] = '-1'

In [ ]:
! pip install 'h5py<3.0.0'


In [ ]:
K.set_learning_phase(0)

## Folder where tf serving weights will be saved
path = './Tensorflow_servings_weights/'

## backbone of mode
backbone = 'resnet101'

## Path to specific location of weights per backbone
path_model_serving = path + backbone
os.makedirs(path_model_serving, exist_ok=True)

### Path to version of model 
export_path = path_model_serving + '/0/'

num_classes = 20
model = models.convert_model(
    model=models.backbone(backbone_name=backbone).retinanet(num_classes=num_classes),
    nms=True,
    class_specific_filter=True,
    anchor_params=None
)


path_checkpoint = '/snapshots/Mark_detection_12k/resnet101_train_lanes_12k.txt_csv_30.h5'
model.load_weights(path_checkpoint)

print('Output layers', [o.name[:-2] for o in model.outputs])
print('Input layer', model.inputs[0].name[:-2])
if os.path.isdir(export_path):
    shutil.rmtree(export_path)

builder = saved_model.builder.SavedModelBuilder(export_path)

signature = predict_signature_def(
    inputs={'input': model.input},
    outputs={
        'output1': model.outputs[0],
        'output2': model.outputs[1],
        'output3': model.outputs[2]
    }
)

## output1 = boxes
## output2 = scores
## output1 = labels

sess = K.get_session()
# sess.run(tf.global_variables_initializer()) 

builder.add_meta_graph_and_variables(sess=sess,
                                     tags=[saved_model.tag_constants.SERVING],
                                     signature_def_map={'predict': signature})
builder.save()
